In [6]:
import json
import math
import re

In [7]:
with open('23599610/23599610_validations.geojson', 'r') as f:
    geojson_data = json.load(f)
with open('23599610/23599610_full_topology_data.geojson', 'r') as f:
    topojson_data = json.load(f)
with open('23599610/23599610_signs.geojson', 'r') as f:
    signjson_data = json.load(f)


In [8]:
def coordcheck(c1,c2,distance_threshold):
    distance = math.sqrt((c2[0] - c1[0])**2 + (c2[1] - c1[1])**2)
    
    # Compare against thresholds
    distance_check = distance < distance_threshold
    return distance_check

In [9]:
topojson_data['features'][0]['properties']['id']

'urn:here::here:Topology:107037225'

In [10]:
for i in range(0,len(geojson_data['features'])):
    signid = geojson_data['features'][i]['properties']['Feature ID']
    ermes = geojson_data['features'][i]['properties']['Error Message']
    topo = re.search(r'urn:here::here:Topology:(\d+)', ermes).group(0)
    print(i)
    print(topo)
    for j in range(0,len(signjson_data['features'])):
        c2 = signjson_data['features'][j]['properties']['id']
        if signjson_data['features'][j]['properties']['signType'] == 'MOTORWAY' and signid == c2:
            print(signjson_data['features'][j]['properties'])
    for x in range(0,len(topojson_data['features'])):
        c3 = topojson_data['features'][x]['properties']['id']
        if topo == c3:
            print(topojson_data['features'][x]['properties'])
            print(x)

0
urn:here::here:Topology:158070718
{'partition_id': '23599610', 'id': 'urn:here::here:signs:1621752874458049432', 'widthCm': {'dimensionCm': 127}, 'heightCm': {'dimensionCm': 78}, 'signType': 'MOTORWAY', 'signValue': '', 'confidence': {'simpleScores': [{'score': 0.2873648047852831, 'sources': ['BMW_RSD'], 'scoreType': 'EXISTENCE', 'featureType': 'SIGN_EXISTENCE', 'positionCovariance': {'values': [0.27374980370180774, 0.03994823025780281, 2.817924011526292, 0.2951580213317938, 2.815995176119427, 3.077577268638773]}, 'existenceProbability': 0.2873648047852831}, {'score': 0.7398462025179068, 'sources': ['BMW_RSD'], 'scoreType': 'CLASSIFICATION', 'featureType': 'SIGN_CLASSIFICATION', 'positionCovariance': {'values': [0.27374980370180774, 0.03994823025780281, 2.817924011526292, 0.2951580213317938, 2.815995176119427, 3.077577268638773]}}, {'score': 0.8259949805388344, 'sources': ['BMW_RSD'], 'scoreType': 'SPATIAL', 'featureType': 'SIGN_GEOMETRY', 'positionCovariance': {'values': [0.27374980

In [11]:
file_path = 'data.json'

with open(file_path, 'w') as json_file:
    json.dump(topojson_data['features'][0]['properties'], json_file, indent=4)

In [12]:
len(topojson_data['features'])

4693